# `Enterprise` Activity for IPTA 2019 Student Week $-$ Pune, India (NCRA/TIFR)

## Using enterprise to do Bayesian analysis


`ENTERPRISE` (Enhanced Numerical Toolbox Enabling a Robust PulsaR Inference SuitE) is a pulsar timing analysis code, aimed at noise analysis, gravitational-wave searches, and timing model analysis. This tutorial is based on the enterprise's documentation, https://enterprise.readthedocs.io/en, the worksheets on IPTA GitHub, https:/github.com/ipta/gwa_tutorials, and the tutorials of last IPTA meeting, https://github.com/ipta/ipta-2018-workshop/tree/master/gw_detection.

You should have learned how to use `TempoNest` and got some experience of Bayesian analysis in pulsar timing. Usually we sample the parameter space with existing Bayesian sampling softwares, like `MultiNest` used by `TempoNest`. We provide the sampler with prior distribution and likelihood function, and the sampler will give us the posterior distrubution. The key part of `TempoNest` and `enterprise` is how to calculate the likelihood. We specify the data, the prior distribution of parameters, and the components of signal model, `TempoNest` will construct the likelihood function and pass it to `Multinest`. `Enterprise` does similar things, but in a more flexible way. It provides the ability to add complicated signals to PTA models, like GW signal, and you can even write your own signal easily. 

In this notebook, we will start with noise analysis of single pulsar. Then you wil learn how to search GW background and GW from individual source. You may know that Bayesian inference is computationally expensive, especially when using PTA to search GWs. So we can only get the results quickly for simple case of single-pulsar noise analysis (also only with limited parameters and data points.) We will show how to post-process the results for this case. For the GW problem, we only show how to setup the analysis and get the likelihood. 

## Load modules

In [ ]:
#from __future__ import division

import numpy as np
import os, glob, json 
import matplotlib.pyplot as plt
import scipy.linalg as sl

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
import enterprise_extensions
from enterprise_extensions import models, model_utils

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

# Part I. Noise analysis of single pulsar
We start with noise analysis of single pulsar, as you have done using `TempoNest`. The likelihood function is determined by the data and the signal model. First, we load the pulsar timing data into `Pulsar` object in `enterprise`.

## Load in `Pulsar` class
`Enterprise` uses a specific `Pulsar` object to store all of the relevant pulsar information (i.e. TOAs, residuals, error bars, flags, etc) from the timing package like `tempo2` or `PINT`. This class is instantiated with a par and a tim file. 


In [ ]:
datadir = enterprise.__path__[0] + '/datafiles/mdc_open1/'
par = datadir+'J0030+0451.par'
tim = datadir+'J0030+0451.tim'
#datadir = enterprise.__path__[0] + '/datafiles/ng9/'
#par=datadir+'J0613-0200_NANOGrav_9yv1.gls.par'
#tim=datadir+'J0613-0200_NANOGrav_9yv1.tim'
psr = Pulsar(par, tim)

The object `psr` contains all data needed for the analysis. You no longer need to reference the `par` and `tim` files after this cell. You can check the content of `psr`, such as name, TOAs and timing residuals. The simulated data have small TOA errors and strong red noise.

In [ ]:
plt.title(psr.name)
plt.errorbar(psr.toas/86400,psr.residuals*1e6,yerr=psr.toaerrs*1e6,fmt='.');
plt.xlabel('t(MJD)')
plt.ylabel('res $(\mu s)$');

## Signal model
Second, we need to set up the signal model. The basic model-building steps in `enterprise` is as following, 
* Define `parameters` and priors;
* Set up the `signals` these `parameters` belong to;
* Define the `model` by summing the individual `signals`;
* Define `PTA` by initializing the signal model with a `Pulsar` object.

For this simulated data, only white noise (measurement noise) and red noise are needed. However, we will also show how to create the parameters and signals for other noise processes. 

### Create parameters
`Parameters` are set by specifying a prior distribution (i.e., uniform, log-uniform, etc.). The parameters include Efac, Equad and Ecorr for white noise, amplitude and spectra index for red noise and DM noise with power-law spectra.


In [ ]:
# white noise parameters
efac = parameter.Uniform(0.01, 10.0)
equad = parameter.Uniform(-8.5, -5)
ecorr = parameter.Uniform(-8.5, -5)

# red noise parameters
log10_A = parameter.Uniform(-20, -11)
gamma = parameter.Uniform(0, 7)

# dm-variation parameters
log10_A_dm = parameter.Uniform(-20, -11)
gamma_dm = parameter.Uniform(0, 7)

### Create signals

Then we create the signal components respectively. 
* `White noise`: The white noise signals corresponding to Efac, Equad and Ecorr are in the `white signals` module with name of `MeasurementNoise`, `EquadNoise` and `EcorrKernelNoise`. Note that different backends have different values of white noise parameters, so we also use `selection` class to split the data by backends and get the white noise parameters for every backend.
* `Red noise`: The red noise signal is in the `gp_signals` module. We need the spectra and basis function to construct red noise. Here we use power-law spectra and Fourier basis. First we create the spectra using `powerlaw` function in `utils` module, and then use the `FourierBasisGP` function to create red noise. We need to specify the frequency sampling by the time span of the data and the number of frequency componnents .
* `DM noise`: The DM noise is similar to red noise, except that it is frequency dependent. We use the `createfourierdesignmatrix_dm` function to create the basis of DM noise.
* `timing model`: We also include timing model, which will be marginalized analytically to make the noise parameter estimation robust.

In [ ]:
# define selection by observing backend
selection = selections.Selection(selections.by_backend)

# special selection for ECORR only use wideband NANOGrav data
selection2 = selections.Selection(selections.nanograv_backends)

# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection2)

# find the maximum time span to set red-noise/DM-variation frequency sampling
tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = np.max(tmax) - np.min(tmin)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

# DM-variations (powerlaw with 30 frequencies)
dm_basis = utils.createfourierdesignmatrix_dm(nmodes=30, Tspan=Tspan)
dm_pl = utils.powerlaw(log10_A=log10_A_dm, gamma=gamma_dm)
dm_gp = gp_signals.BasisGP(dm_pl, dm_basis, name='dm_gp')

# timing model
tm = gp_signals.TimingModel(use_svd=True)

### Piece the full model together

We add the signal components together to create the signal model. The combination of signal model and data to calculate the likelihood  is achieved in the `PTA` class. So we initialize the `model` calss with `psr` object, and pass it to the `PTA` class to create `pta` object (with one pulsar only). We add ef, rn and tm for this simulated data set.

In [ ]:
# intialize a single-pulsar pta model
# model=ef + eq + ec + rn + dm
model = ef + rn  + tm
    
pta = signal_base.PTA(model(psr))

Now we have finished constructing the signal model. We can look into the `pta` object to see which parameters we are going to be searching over with.

In [ ]:
for par in pta.params:
    print(par)

We will start the MCMC chain at a random point in parameter space. We accomplish this by setting up a parameter dictionary using the name and sample methods for each Parameter. We can check the likelihood and prior at this point.

In [ ]:
xs = {par.name: par.sample() for par in pta.params}
print(pta.get_lnlikelihood(xs));
print(pta.get_lnprior(xs));

### Set up sampler 
The rest of the analysis here is dependent on the sampling method and not on `enterprise` itself. Traditionally `enterprise` uses PTMCMCSampler package for sampling, but you can also use other samplers, such as `MultiNest` used in `TempoNest`. 
For PTMCMCSampler, as many others, it requires a function to compute the log-likelihood and log-prior given a vector of parameters. Here, these are supplied by `PTA` class as `pta.get_lnlikelihood` and `pta.get_lnprior`.

In [ ]:
# initial jump covariance matrix
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)
cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution

# where chains will be written to
outdir = './chains/noise_run_{}/'.format(str(psr.name))

# sampler object
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,
                 outDir=outdir, 
                 resume=False)

### Sample the parameter space

The time taken by sampling depends on your laptop. You will see printout like "Finished aa percent in bb s". You can estimate the time needed, which should be several minutes. If it is too long, you can stop sampling by "Kernel->Interrupt" and jump to Part II.

In [ ]:
# sampler for N steps
#N=int(1e6)
N = int(5e4)

# SCAM = Single Component Adaptive Metropolis
# AM = Adaptive Metropolis
# DE = Differential Evolution
## You can keep all these set at default values
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

### Simple post-processing
Now we read the output of sampling and plot the posterior distribution of parameters. The histograms show the marginalized posterior distrubution of every parameter, and the contour plots show the 2-d posterior distribution of parameter pairs, from which we can see the correlation between parameters.

In [ ]:
chain = np.loadtxt(outdir + 'chain_1.txt')
burn = int(0.25 * chain.shape[0])
corner.corner(chain[burn:,:-4],labels=pta.param_names,levels=[0.68,0.95]);

# Part II. Stochastic gravitational wave background

GWB analysis is very similar to the noise analysis above, except that we need to analyze multiple pulsars simultaneously to search for spatial correlation between pulsars. Another main difference is that we add GWB signal to the signal model.

## Get par, tim, and noise files
Here we collect the tim and par files as well as noise files. These are the same par, tim, and noise files used in the NANOGrav 9-year analysis papers. We need noise files because we want to fix white noise parameters at the values from single-pulsar noise analysis. 

In [ ]:
datadir = enterprise.__path__[0] + '/datafiles/ng9/'

parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))
noisefiles = sorted(glob.glob(datadir + '/*noise.txt'))

# 18 pulsars used in 9 year analysis
p9 = np.loadtxt(datadir+'/9yr_pulsars.txt', dtype=str)#'S42')

# filter
parfiles = [x for x in parfiles if x.split('/')[-1].split('_')[0] in p9]
timfiles = [x for x in timfiles if x.split('/')[-1].split('_')[0] in p9]
noisefiles = [x for x in noisefiles if x.split('/')[-1].split('_')[0] in p9]

When you execute the following cell, you may get warnings like `WARNING: Could not find pulsar distance for PSR ...`. Don't worry! This is expected, and fine. Not all pulsars have well constrained distances, and will be set to `1 kpc` with a `20%` uncertainty.

In [ ]:
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem='DE421')
    psrs.append(psr)

## Signal model
The basic difference in signal model is that we need to create the GWB signal. The GWB signal is also stochastic with power-law spectra, but with spatial correlation described by Hellings-Downs curve. Note that GWB parameters are common between different pulsars. We initialize GW parameters with name, so that these parameters will be shared among pulsars, rather than create one set of parameters for every pulsar.
The spectra index of GWB is 13/3, so we fix this parameter. 

In [ ]:
# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_A_gw = parameter.LinearExp(-18,-12)('log10_A_gw')
gamma_gw = parameter.Constant(4.33)('gamma_gw')

# gwb
cpl = utils.powerlaw(log10_A=log10_A_gw, gamma=gamma_gw)
orf = utils.hd_orf()
gw = gp_signals.FourierBasisCommonGP(cpl, orf, components=30, Tspan=Tspan, name='gw')

The rest of the signal model is created in a similar way to noise analysis, but with multiple pulsars. In search for GW, we usually fix the white noise parameters at the value from single-pulsar noise analysis. We can also add the modelling of errors in solar system ephemeris to the signal model, using `PhysicalEphemerisSignal` function in `deterministic_signals` module.

In [ ]:
# find the maximum time span to set GW frequency sampling
tmin = [p.toas.min() for p in psrs]
tmax = [p.toas.max() for p in psrs]
Tspan = np.max(tmax) - np.min(tmin)

# selection class to break white noise by backend
selection = selections.Selection(selections.by_backend)

##### parameters and priors #####

# white noise parameters
# since we are fixing these to values from the noise file we set
# them as constant parameters
efac = parameter.Constant()
equad = parameter.Constant()
ecorr = parameter.Constant()

# red noise parameters 
log10_A = parameter.LinearExp(-20,-12)
gamma = parameter.Uniform(0,7)


##### Set up signals #####

# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

# timing model
tm = gp_signals.TimingModel()

# to add solar system ephemeris modeling...
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)

# full model is sum of components
model = ef + eq + ec + rn + tm + eph + gw

# intialize PTA
pta = signal_base.PTA([model(psr) for psr in psrs])

Set values of white noise parameters from noise files.

In [ ]:
#convert `PAL2` noise files to `enterprise` parameter dictionaries.
def get_noise_from_pal2(noisefile):
    psrname = noisefile.split('/')[-1].split('_noise.txt')[0]
    fin = open(noisefile, 'r')
    lines = fin.readlines()
    params = {}
    for line in lines:
        ln = line.split()
        if 'efac' in line:
            par = 'efac'
            flag = ln[0].split('efac-')[-1]
        elif 'equad' in line:
            par = 'log10_equad'
            flag = ln[0].split('equad-')[-1]
        elif 'jitter_q' in line:
            par = 'log10_ecorr'
            flag = ln[0].split('jitter_q-')[-1]
        elif 'RN-Amplitude' in line:
            par = 'log10_A'
            flag = ''
        elif 'RN-spectral-index' in line:
            par = 'gamma'
            flag = ''
        else:
            break
        if flag:
            name = [psrname, flag, par]
        else:
            name = [psrname, par]
        pname = '_'.join(name)
        params.update({pname: float(ln[1])})
    return params

params = {}
for nfile in noisefiles:
    params.update(get_noise_from_pal2(nfile))
    
pta.set_default_params(params)

Now we have set up the model. We can check the parameters and the likelihood to see if everything works correctly. 

In [ ]:
for par in pta.params:
    print(par)

In [ ]:
xs = {par.name: par.sample() for par in pta.params}
print(pta.get_lnlikelihood(xs));
print(pta.get_lnprior(xs));

### Basic idea of GWB analysis
The rest of work is left to the Bayesian sampler. This part is computationally expensive, and is impossible to finish here. We just introduce some basic ideas of GWB analysis. 
* To answer whether we have detected GWB, we need compare models with and without GWB signal. We can only claim detection of GWB when the model with GWB has significantly larger Bayesian evidence. 
* If we don't detect GWB, we can set an upper limit on its amplitude. Note that for the amplitude, we use uniform prior in upper limit analysis, and log-uniform prior in searching.

# Part III. Gravitational wave from single source

GWB is thought to be the most likely source to be detected by PTA. However, It's also possible to detect continuous gravitational wave (CGW) from individual supermassive blackhole binary, if the signal is strong enough. The CGW signal is deterministic, which depends on the mass, orbital parameters, and position of the binary. CGW analysis uses the same data as GWB analysis, so we only need to deal with the signal model. In the following, we first create CGW parameters, then calculate the CGW waveform, and create the CGW signal. Note that we are initializing them with names here so that they will be shared across all pulsars in the PTA.

In [ ]:
# continuous GW parameters

cos_gwtheta = parameter.Uniform(-1, 1)('cos_gwtheta')
gwphi = parameter.Uniform(0, 2*np.pi)('gwphi')
log10_mc = parameter.Uniform(7, 10)('log10_mc')
log10_fgw = parameter.Uniform(-10,-7)('log10_fgw')
phase0 = parameter.Uniform(0, 2*np.pi)('phase0')
psi = parameter.Uniform(0, np.pi)('psi')
cos_inc = parameter.Uniform(-1, 1)('cos_inc')

log10_h = parameter.LinearExp(-18, -11)('log10_h')

# define CGW waveform and signal
cw_wf = models.cw_delay(cos_gwtheta=cos_gwtheta, gwphi=gwphi, log10_mc=log10_mc, 
                     log10_h=log10_h, log10_fgw=log10_fgw, phase0=phase0, 
                     psi=psi, cos_inc=cos_inc)
cw = models.CWSignal(cw_wf, psrTerm=True)

We can replace the GWB signal in the previous part with CGW signal, and create the model and PTA object.

In [ ]:
# full model is sum of components
model = ef + eq + ec + rn + tm + cw

# intialize PTA
pta = signal_base.PTA([model(psr) for psr in psrs])

In [ ]:
# set values of white noise parameters
params = {}
for nfile in noisefiles:
    params.update(get_noise_from_pal2(nfile))
    
pta.set_default_params(params)

Like the case of GWB analysis, we also stop at checking the parameters and evaluating the likelihood.

In [ ]:
for par in pta.params:
    print(par)

In [ ]:
xs = {par.name: par.sample() for par in pta.params}
print(pta.get_lnlikelihood(xs));
print(pta.get_lnprior(xs));

### Basic idea in CGW analysis
* If we want to quantify whether we have detected CGW, we need to do model selection by comparing the Bayesian evidence of models with and without CGW signal. 
* If we don't detect the signal, we can also set an upper limit on the CGW amplitude. The upper limit is usually given as a function of frequency. To do this, we fix the frequency parameter, and perform Bayesian sampling for the rest of parameters to get the upper limit of GW amplitude. Repeat this process for a series of frequencies, we can get the CGW upper limit at different frequencies.